In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
from matplotlib.gridspec import GridSpec
import orjson
import gzip
import paramiko
from scp import SCPClient
import os
# import subprocess

warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
raw_path = './../../data/Raw/moe_data'
derived_path = './../../data/Derived/moe_data'

In [4]:
password = "xx"

source = "carl.luddy.indiana.edu"
file = '1.gz'
destination = 'xx'
destination_path = '/u/potem/Projects/TweetDeletion/data/Derived'
user='potem'
location = '/l/cnets/research/potem_compliance/extra'

# key = paramiko.RSAKey.from_private_key_file(public_key)

ssh = paramiko.SSHClient()

ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname=source, 
            username=user, 
            password=password)
command = 'cd '+ location + '; '+ 'ls | wc -l'
(stdin, stdout, stderr) = ssh.exec_command(command)

no_of_files = int(stdout.readlines()[0])

scp = SCPClient(ssh.get_transport())

In [5]:
print(no_of_files)

795


In [53]:
# scp.close()

Example moe tweet object

In [ ]:
# '''

# {
#     "favorited":false,
#     "text":"",
#     "created_at":"Sun Apr 25 20:00:07 -0400 2021",
#     "user_id_str":"",
#     "user_screen_name":"",
#     "retweet_count":0,
#     "retweeted":false,
#     "source":"",
#     "id_str":"",
#     "entities":{
#     "hashtags":[
#     ],
#     "urls":[
#     ],
#     "user_mentions":[
#     ],
#     "symbols":[
#     ]
#     },
#     "user":{
#         "created_at":"Tue Feb 12 22:47:58 -0500 2019",
#         "profile_link_color":"1DA1F2",
#         "default_profile":true,
#         "favourites_count":27725,
#         "friends_count":1167,
#         "profile_sidebar_border_color":"C0DEED",
#         "url":"",
#         "profile_use_background_image":true,
#         "description":"",
#         "profile_background_color":"F5F8FA",
#         "followers_count":1577,
#         "profile_image_url":"",
#         "listed_count":11,
#         "geo_enabled":false,
#         "profile_background_image_url":"",
#         "contributors_enabled":false,
#         "verified":false,
#         "screen_name":"",
#         "id_str":"",
#         "profile_text_color":"333333",
#         "is_protected":false,
#         "is_translator":false,
#         "statuses_count":34709,
#         "profile_sidebar_fill_color":"DDEEF6",
#         "location":"3𝖗𝖆𝖈𝖍𝖆",
#         "name":"Bel",
#         "default_profile_image":false,
#         "profile_background_tile":false
#     },
#     "truncated":false,
#     "otherfields":{
#         "entities":"{hashtags:[],urls:[],user_mentions:[],symbols:[]}",
#         "favorite_count":"0",
#         "filter_level":"low",
#         "qd":"\u0000",
#         "quote_count":"0",
#         "reply_count":"0",
#         "rm":"\u0000\u0000\u0000\u0000",
#         "text":"",
#         "timestamp_ms":"1619395207306"
#     }
# }

# '''

In [ ]:
for i in range(1, no_of_files+1):
    #removing all raw files
    for f in os.listdir(raw_path):
        os.remove(os.path.join(raw_path, f))
    
#     #check if cleaned files already exit
    if os.path.exists(f'{derived_path}/{i}.pkl.gz') == True:
        continue
        
# #     files = [location + '/' + str(k)+'.gz' for k in range(i, j)]
    files = location + '/' + str(i)+'.gz'
    
    scp.get(files, raw_path)

    for filename in Path(raw_path).glob('*'):
        name = (str(filename).split('/')[-1]).split('.')[0]
        
        all_data = []
        
        with gzip.open(filename, 'rb') as f:
            data = f.readlines()
            f.close()

            for obj in data:
                k = orjson.loads(obj)

                timestamp = 0
                if 'timestamp_ms' in k['otherfields']:
                    timestamp = k['otherfields']['timestamp_ms']

                all_data.append([k['user_id_str'],
                                k['id_str'],
                                timestamp,
                                k['user']['statuses_count'],
                                k['created_at']])


    df = pd.DataFrame(all_data, columns=['user_id_str', 
                               'id_str', 
                              'timestamp_ms',
                              'statuses_count',          
                              'created_at'])
    df = df.drop_duplicates(subset='id_str', keep='first')
    df['datetime'] = pd.to_datetime(df['created_at'])
    df['date'] = pd.to_datetime(df['datetime'], format='%Y%m%d').dt.tz_localize(None).dt.date
    df = df.sort_values('datetime')
    g = df.groupby(['user_id_str', 'date'])

    df_subset = pd.concat([g.head(1), 
                           g.tail(1)]).drop_duplicates().reset_index(drop=True)
#     df_subset['count'] = df_subset.groupby(['user_id_str', 'date'])['user_id_str'].transform('count')
#     df_subset = df_subset.loc[df_subset['count'] != 1]
    
    df_subset.to_pickle(f'{derived_path}/{i}.pkl.gz')
    
    print(i)

### Concat all the collected data

In [ ]:
moe_data = 'moe_data'
derived_path = './../../data/Derived'

moe_data_path = derived_path + '/' + moe_data 


df_all = pd.DataFrame(columns=['user_id_str', 
                               'id_str', 
                              'timestamp_ms',
                              'statuses_count',          
                              'created_at'])

for file in Path(moe_data_path).glob('*'):
    df = pd.read_pickle(file)
    df_all = df_all.append(df, ignore_index=True)
    
    print(file)

In [ ]:
df_all.to_pickle(f'{derived_path}/moe_data_aggregated.pkl.gz')

In [ ]:
df_all.info()

### Script to rename folders in carl

In [ ]:
# import os

# FList = os.listdir(os.getcwd())

# m = 796
# path1 = '/l/cnets/research/potem_compliance/extra/'
# for i in FList:
#     fileExtension = os.path.splitext(i)[-1]
#     print(fileExtension)
#     print(str(m)+fileExtension)
#     os.rename(os.path.join(path1,i),
#     os.path.join(path1,str(m)+fileExtension))
#     m = m+1

Parse for deviation in deletion count

In [6]:
derived_path = './../../data/Derived'

df_user = pd.read_pickle(f'{derived_path}/deviation.pkl.gz')

df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11411 entries, 0 to 11410
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uid     11411 non-null  object
dtypes: object(1)
memory usage: 89.3+ KB


In [7]:
df_user.head()

,uid
0,1161776053913677824
1,829611048038694913
2,176230580
3,822402741599342592
4,1286282822890614785


In [9]:
# if '1161776053913677824' not in df_user['uid']:
#     print('ok')

# print(type(k['user_id_str']))

In [ ]:
all_data = []
derived_path = './../../data/Derived'
raw_path = './../../data/Raw/moe_data'
user_ids = df_user['uid'].values

for i in range(1, no_of_files+1):
    #removing all raw files
    
#     if i <= 106:
#         continue
        
    for f in os.listdir(raw_path):
        os.remove(os.path.join(raw_path, f))
    
#     #check if cleaned files already exit
#     if os.path.exists(f'{derived_path}/{i}.pkl.gz') == True:
#         continue
        
# #     files = [location + '/' + str(k)+'.gz' for k in range(i, j)]
    files = location + '/' + str(i)+'.gz'
    
    scp.get(files, raw_path)

    for filename in Path(raw_path).glob('*'):
        name = (str(filename).split('/')[-1]).split('.')[0]
        
        with gzip.open(filename, 'rb') as f:
            data = f.readlines()
            f.close()

            for obj in data:
                k = orjson.loads(obj)
                
                if k['user_id_str'] not in user_ids:
                    continue

                timestamp = 0
                if 'timestamp_ms' in k['otherfields']:
                    timestamp = k['otherfields']['timestamp_ms']

                all_data.append([k['user_id_str'],
                                k['id_str'],
                                timestamp,
                                k['user']['statuses_count'],
                                k['created_at']])
        print(len(all_data)) 
        print('file : ', name)
                


df = pd.DataFrame(all_data, columns=['user_id_str', 
                           'id_str', 
                          'timestamp_ms',
                          'statuses_count',          
                          'created_at'])
df = df.drop_duplicates(subset='id_str', keep='first')
df['datetime'] = pd.to_datetime(df['created_at'])
df['date'] = pd.to_datetime(df['datetime'], format='%Y%m%d').dt.tz_localize(None).dt.date

df.to_pickle(f'{derived_path}/deviation_raw.pkl.gz')

9521
file :  1
12098
file :  2
13802
file :  3
15563
file :  4
16226
file :  5
25340
file :  6
25722
file :  7
41490
file :  8
44824
file :  9
45514
file :  10
46523
file :  11
48502
file :  12
57501
file :  13
70781
file :  14
75612
file :  15
79233
file :  16
80210
file :  17
88118
file :  18
92431
file :  19
94888
file :  20
96149
file :  21
105177
file :  22
105300
file :  23
115736
file :  24
119366
file :  25
120279
file :  26
123973
file :  27
126169
file :  28
131477
file :  29
132436
file :  30
135297
file :  31
141036
file :  32
147723
file :  33
148754
file :  34
149396
file :  35
151048
file :  36
151823
file :  37
155033
file :  38
163678
file :  39
166933
file :  40
171983
file :  41
179157
file :  42
182786
file :  43
183962
file :  44
192380
file :  45
196544
file :  46
197887
file :  47
199138
file :  48
200394
file :  49
201912
file :  50
204378
file :  51
210619
file :  52
211137
file :  53
211652
file :  54
216611
file :  55
226612
file :  56
227233
file :  57
23135

In [16]:
df = pd.read_pickle(f'{derived_path}/deviation_raw.pkl.gz')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_id_str     0 non-null      object        
 1   id_str          0 non-null      object        
 2   timestamp_ms    0 non-null      object        
 3   statuses_count  0 non-null      object        
 4   created_at      0 non-null      object        
 5   datetime        0 non-null      datetime64[ns]
 6   date            0 non-null      object        
dtypes: datetime64[ns](1), object(6)
memory usage: 0.0+ bytes
